## This notebook calculate LECIF scores for genes

In [1]:
import os

os.environ['SPECIES'] = "Mouse"
from eval.visual_utils import (get_naming_dict, filter_dataframe, FigureStyle, pval2stars, Naming_Json, export_to_excel)
njson = Naming_Json()
exp = njson.exp
exp_short = exp.split("_")[0]
m_param_json_file = f"/g/data/yr31/zs2131/tasks/2023/RNA_expr_net/anal_mouse/training/{exp_short}/{exp}.param_config.json"
os.environ['PARAM_JSON_FILE'] = m_param_json_file
from utils.params import params
from utils.utils import get_config
import statistics
import gzip
params = params()
config = get_config()

Using mouse project folder: /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/GeneRAIN_Mouse
Using mouse project folder: /g/data/yr31/zs2131/tasks/2023/RNA_expr_net/GeneRAIN_Mouse


In [2]:
# Dictionary to store CDS regions for each gene, including chromosome info

feature_gene_regions = {}
features_of_interest = ['CDS', 'gene']
for feature in features_of_interest:
    feature_gene_regions[feature] = {}
# Open and read the compressed GFF3 file
with gzip.open(f"{config.proj_path}/data/external/Gencode/gencode.v44.annotation.gff3.gz", "rt") as gff3_file:
    for line in gff3_file:
        if line.strip() and not line.startswith("#"):
            parts = line.strip().split("\t")
            feature = parts[2]
            if feature in features_of_interest:
                gene_regions = feature_gene_regions[feature]
                chrom = parts[0]
                gene_info = parts[8]
                #gene_id = [info for info in gene_info.split(";") if info.startswith("gene_id=")][0].split("=")[1]
                gene_id = [info for info in gene_info.split(";") if info.startswith("gene_name=")][0].split("=")[1]
                start = int(parts[3])
                end = int(parts[4])
                
                if gene_id not in gene_regions:
                    gene_regions[gene_id] = []
                gene_regions[gene_id].append((chrom, start, end))

gene_cds = feature_gene_regions['CDS']
gene_wholeGene = feature_gene_regions['gene']
# remove duplicates
gene_cds = {gene_id: list(set(regions)) for gene_id, regions in gene_cds.items()}
gene_wholeGene = {gene_id: list(set(regions)) for gene_id, regions in gene_wholeGene.items()}

In [3]:
gene_cds_values = {}
gene_wholeGene_values = {}


In [4]:
# to deal with memory constraint
for target_chr in [str(i) for i in range(1, 23)] + ["X", "Y"]:
# for target_chr in ["21", "22"]:
    print(f"analyzing chr{target_chr}")
    wig_data = {}
    # Parse the WIG file and calculate the average for each gene with chromosome matching
    with open(f"{config.proj_path}/data/external/LECIF/hg38.LECIFv1.1.wig", "r") as wig_file:
        i = 0
        for line in wig_file:
            i += 1
            # if i % 500000 == 0:
            #     print(f"Finished line {i}")
            if line.strip():
                if "#" in line:
                    continue
                chrom, start, end, value = line.strip().split()

                chrom = chrom.replace("chr", "")  # Adjusting chrom format if needed
                if target_chr != chrom:
                    continue
                start, end, value = int(start), int(end), float(value)
                for pos in range(start, end + 1):
                    key = f"{chrom}:{pos}"
                    wig_data[key] = value
    
    def calculate_average_coverage(gene_cds, wig_data, gene_values, target_chr):
        for gene_id, regions in gene_cds.items():
            total_value, total_positions = 0, 0
            value_list = []
            # to saved the pos of checked pos, to avoid a same pos checked multiple times for a gene
            checked = {}
            for chrom, start, end in regions:
                chrom = chrom.replace("chr", "")  # Adjusting chrom format
                if target_chr != chrom:
                    continue
                
                for pos in range(start, end + 1):
                    key = f"{chrom}:{pos}"
                    if key in checked:
                        continue
                    checked[key] = 1
                    if key in wig_data:
                        total_value += wig_data[key]
                        total_positions += 1
                        value_list.append(wig_data[key])
            
            # Avoid division by zero if there are no positions found in wig_data
            if value_list:
                average = statistics.mean(value_list)
                std_dev = statistics.stdev(value_list) if len(value_list) > 1 else 0
                median = statistics.median(value_list)
                gene_values[gene_id] = {
                    "average": average,
                    "std_dev": std_dev,
                    "median": median,
                    "total_n": len(value_list)
                }
        return gene_values
    
    gene_cds_values = calculate_average_coverage(gene_cds, wig_data, gene_values=gene_cds_values, target_chr=target_chr)
    gene_wholeGene_values = calculate_average_coverage(gene_wholeGene, wig_data, gene_values=gene_wholeGene_values, target_chr=target_chr)


analyzing chr1
analyzing chr2
analyzing chr3
analyzing chr4
analyzing chr5
analyzing chr6
analyzing chr7
analyzing chr8
analyzing chr9
analyzing chr10
analyzing chr11
analyzing chr12
analyzing chr13
analyzing chr14
analyzing chr15
analyzing chr16
analyzing chr17
analyzing chr18
analyzing chr19
analyzing chr20
analyzing chr21
analyzing chr22
analyzing chrX
analyzing chrY


In [5]:
import csv

# Assuming gene_cds_values and gene_wholeGene_values are dictionaries
output_file = f"{config.proj_path}/results/anal/stat/LECIF_stat.tsv"

with open(output_file, "w", newline="") as tsvfile:
    writer = csv.writer(tsvfile, delimiter="\t")
    
    # Write the header row
    writer.writerow(["GeneID", "CDS_Average", "CDS_StdDev", "CDS_Median", "CDS_Total_Pos_with_LECIF",
                     "Gene_Average", "Gene_StdDev", "Gene_Median", "Gene_Total_Pos_with_LECIF"])
    
    # Get the union of keys from both dictionaries
    all_genes = set(gene_cds_values.keys()) | set(gene_wholeGene_values.keys())
    
    # Iterate through all genes and write their statistics
    for gene_id in all_genes:
        # Extract CDS values or set defaults if the gene_id is not in gene_cds_values
        cds_values = gene_cds_values.get(gene_id, {"average": "NA", "std_dev": "NA", "median": "NA", "total_n": "NA"})
        cds_average = cds_values.get("average", "NA")
        cds_std_dev = cds_values.get("std_dev", "NA")
        cds_median = cds_values.get("median", "NA")
        cds_total_n = cds_values.get("total_n", "NA")
        
        # Extract whole gene values or set defaults if the gene_id is not in gene_wholeGene_values
        whole_gene_values = gene_wholeGene_values.get(gene_id, {"average": "NA", "std_dev": "NA", "median": "NA", "total_n": "NA"})
        whole_gene_average = whole_gene_values.get("average", "NA")
        whole_gene_std_dev = whole_gene_values.get("std_dev", "NA")
        whole_gene_median = whole_gene_values.get("median", "NA")
        whole_gene_total_n = whole_gene_values.get("total_n", "NA")
        
        # Write the row for this gene
        writer.writerow([gene_id, cds_average, cds_std_dev, cds_median, cds_total_n,
                         whole_gene_average, whole_gene_std_dev, whole_gene_median, whole_gene_total_n])
